In [1]:
! pip install transformers==4.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 36.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.6 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=b66a2bfad13487d19624b6aaa36f5139f43184768db469022a683ceb8fb4be23
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses


# **Data**

In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 8.0 MB/s eta 0:00:00


# Emo Event dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


emoevent_train_csv_path = '/content/drive/MyDrive/Colab Notebooks/datasets/emoevent_train.csv'
emoevent_dev_csv_path = '/content/drive/MyDrive/Colab Notebooks/datasets/emoevent_test.csv'

emoevent_train = pd.read_csv(emoevent_train_csv_path)
emoevent_dev = pd.read_csv(emoevent_dev_csv_path)


In [5]:
print('emoevent_train shape:', emoevent_train.shape)
emoevent_train['tweet'].head(10)

emoevent_train shape: (5112, 2)


0    What is one thing that you can not live withou...
1    Hahahhaha bells will be ringing to show solida...
2    The Compassionate Civilization Collaborative (...
3    They say much was spared, but the images of HA...
4    "I may be small. I may be a girl, but I won’t ...
5    HASHTAG Today we have begun the definitive pha...
6    "The highest education is that which does not ...
7    “Hi Friends! Lots of people are making comment...
8    HASHTAG election: socialist party HASHTAG decl...
9    THAT EPISODE WAS FUCKING EVERYTHING... 🤯🤯🤯🤯🤯 H...
Name: tweet, dtype: object

In [6]:
emoevent_train['emotion'].head(10)

0    4
1    0
2    2
3    5
4    4
5    3
6    4
7    4
8    4
9    3
Name: emotion, dtype: int64

In [7]:
X_train = emoevent_train['tweet']
y_train = emoevent_train['emotion']

X_test = emoevent_dev['tweet']
y_test = emoevent_dev['emotion']

In [8]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5112,)
(5112,)
(2191,)
(2191,)


In [9]:
# label: 'sadness': 0, 'neutral': 6, 'fear': 2, 'anger': 3, 'disgust': 4, 'surprise': 5, 'joy': 6
y_train.value_counts()

4    2313
3    1427
1     536
5     291
0     274
6     165
2     106
Name: emotion, dtype: int64

In [10]:
y_test.value_counts()

4    992
3    612
1    229
5    125
0    118
6     70
2     45
Name: emotion, dtype: int64

# **Model**


In [11]:
# Fit and transform X_train using Tfidf Vectorizer with default parameters
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
print('tfidf train shape:', X_train_tfidf.shape)
print('tfidf train type:', X_train_tfidf.dtype)

X_test_tfidf = vectorizer.transform(X_test).toarray()
print('tfidf test:', X_test_tfidf.shape)


tfidf train shape: (5112, 9912)
tfidf train type: float64
tfidf test: (2191, 9912)


In [12]:
from sklearn.svm import LinearSVC
clf = LinearSVC().fit(X_train_tfidf, y_train)

from sklearn import metrics
predicted = clf.predict(X_test_tfidf)

acc = metrics.accuracy_score(y_test, predicted)
print('accuracy is: ', acc*100)

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,predicted))
print('\n')
print(classification_report(y_test,predicted))

accuracy is:  53.40027384755819
[[  5  21   0   6  83   2   1]
 [ 14  48   2  21 138   4   2]
 [  1   3   1   3  36   1   0]
 [  0   7   0 354 239   8   4]
 [ 19  47   3 194 712  15   2]
 [  4   8   0  12  53  47   1]
 [  1   3   0  15  47   1   3]]


              precision    recall  f1-score   support

           0       0.11      0.04      0.06       118
           1       0.35      0.21      0.26       229
           2       0.17      0.02      0.04        45
           3       0.59      0.58      0.58       612
           4       0.54      0.72      0.62       992
           5       0.60      0.38      0.46       125
           6       0.23      0.04      0.07        70

    accuracy                           0.53      2191
   macro avg       0.37      0.28      0.30      2191
weighted avg       0.50      0.53      0.50      2191



# Augmentation using Embedding Word2Vec

In [13]:
! pip install nlpaug==1.1.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 6.2 MB/s eta 0:00:00


In [14]:
from nlpaug.util.file.download import DownloadUtil
DownloadUtil.download_glove('glove.6B', '.')

In [15]:
# apply augmentation to the train data and save the results into a file
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf

from nlpaug.util import Action
import nlpaug.flow as naf

aug_emb = naf.Sequential([naf.Sometimes([
    naw.WordEmbsAug(
    model_type='glove', model_path='./glove.6B.200d.txt',
    action="substitute")]),
    naf.Sometimes([naw.WordEmbsAug(
    model_type='glove', model_path='./glove.6B.200d.txt',
    action="insert"),
])])


In [16]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [17]:
from tqdm.notebook import tqdm
tqdm.pandas()

data = {'text': X_train, 'label': y_train}
base_train = pd.DataFrame.from_dict(data)
base_train.shape

(5112, 2)

In [18]:
rep = 5 # how many repetitions of the record


emb_train = base_train.copy()
emb_train['paraphrase'] = emb_train['text'].progress_apply(lambda x:aug_emb.augment(x, rep))
emb_train = emb_train.explode('paraphrase').reset_index(drop=True)

  0%|          | 0/5112 [00:00<?, ?it/s]

In [19]:
emb_train.head(20)

,text,label,paraphrase
0,What is one thing that you can not live withou...,4,1.5550 What is one cimahi thing vachel that yo...
1,What is one thing that you can not live withou...,4,What is one marse thing beakers that you can n...
2,What is one thing that you can not live withou...,4,What is 562.5 one thing that you can kreiensen...
3,What is one thing that you can not live withou...,4,What is muhajaria one originations thing frye ...
4,What is one thing that you can not live withou...,4,What is one something result you could not liv...
5,Hahahhaha bells will be ringing to show solida...,0,Hahahhaha sound hoped blitz be flashing to sho...
6,Hahahhaha bells will be ringing to show solida...,0,gastroenterologist Hahahhaha tinkling. be ring...
7,Hahahhaha bells will be ringing to show solida...,0,Hahahhaha bagpipes will be ringing to show bro...
8,Hahahhaha bells will be ringing to show solida...,0,Hahahhaha carillon enough orelli be ringing to...
9,Hahahhaha bells will be ringing to show solida...,0,Hahahhaha bells clasica will must shushed peal...


# Model with augmented train data (paraphrased data)

In [20]:
# Fit and transform X_train using Tfidf Vectorizer with default parameters
from sklearn.feature_extraction.text import TfidfVectorizer

# !! we should concatenate original train with paraphrased train
new_data = X_train.append(emb_train['paraphrase'])

vectorizer = TfidfVectorizer() #max_features=3000)
X_train_tfidf = vectorizer.fit_transform(new_data).toarray() #emb_train['paraphrase']).toarray()
print('tfidf train shape:', X_train_tfidf.shape)
print('tfidf train type:', X_train_tfidf.dtype)

X_test_tfidf = vectorizer.transform(X_test).toarray()
print('tfidf test:', X_test_tfidf.shape)


<ipython-input-20-c3906541702a>:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = X_train.append(emb_train['paraphrase'])


tfidf train shape: (30672, 109556)
tfidf train type: float64
tfidf test: (2191, 109556)


In [21]:
from sklearn.svm import LinearSVC
# !! we should concatenate original labels with paraphrased text labels
new_labels = y_train.append(emb_train['label'])

clf = LinearSVC().fit(X_train_tfidf, new_labels) # emb_train['label'])

from sklearn import metrics
predicted = clf.predict(X_test_tfidf)

acc = metrics.accuracy_score(y_test, predicted)
print('accuracy is: ', acc*100)

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,predicted))
print('\n')
print(classification_report(y_test,predicted))

<ipython-input-21-01ce137b73dd>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_labels = y_train.append(emb_train['label'])


accuracy is:  50.66179826563213
[[  9  23   0   9  73   3   1]
 [ 18  55   4  25 117   7   3]
 [  2   2   2   0  37   1   1]
 [  2   8   1 373 209  15   4]
 [ 31  68  12 226 613  28  14]
 [  4  12   2   7  43  55   2]
 [  4   7   0  16  38   2   3]]


              precision    recall  f1-score   support

           0       0.13      0.08      0.10       118
           1       0.31      0.24      0.27       229
           2       0.10      0.04      0.06        45
           3       0.57      0.61      0.59       612
           4       0.54      0.62      0.58       992
           5       0.50      0.44      0.47       125
           6       0.11      0.04      0.06        70

    accuracy                           0.51      2191
   macro avg       0.32      0.30      0.30      2191
weighted avg       0.48      0.51      0.49      2191



# Create CSV file from augmented dataset

In [22]:
!cd '/content/drive/My Drive/Colab Notebooks/datasets/'
emb_train.to_csv('/content/drive/My Drive/Colab Notebooks/datasets/emoevent_embaug_5.csv', encoding='utf-8', index=False, sep=',')